#### situation
there are nodes which can connect together as neighbors. each node has a subset of keys (identifiers of it's direct neighbors) & values (information like communication channel & method); the big picture is: there is an unevenly distributed associative array containing all the information about the whole network that our nodes have created together.

#### problem
this distributed associative array (or dictionary or hashmap or whatever you want to call it) is going to be used by all the nodes to discover eachother. and we need some kind of basic implementation of it.

In [18]:
# interface
from __future__ import annotations
from abc import ABC, abstractmethod
from enum import Enum


class ChannelType(Enum):
    HTTP = 1
    GRPC = 2


class INode(ABC):
    @property
    @abstractmethod
    def identifier(self) -> str: ...

    @property
    @abstractmethod
    def public_key(self) -> str: ...

    @property
    @abstractmethod
    def channel(self) -> tuple[ChannelType, str, str]: ...

    @abstractmethod
    def connect(self, neighbor: INode) -> None: ...

    @abstractmethod
    def find(self, questioners: set[str], subject: str) -> INode: ...

In [19]:
# implemnetation
class Node(INode):
    def __init__(self, identifier: str, public_key: str, channel: tuple[ChannelType, str, str]) -> None:
        self._identifier: str = identifier
        self._public_key: str = public_key
        self._channel: tuple[ChannelType, str, str] = channel
        self._neighbors: dict[str, INode] = dict()

    @property
    def identifier(self) -> str:
        return self._identifier

    @property
    def public_key(self) -> str:
        return self._public_key

    @property
    def channel(self) -> tuple[ChannelType, str, str]:
        return self._channel

    def connect(self, neighbor: INode) -> None:
        if neighbor.identifier in self._neighbors.keys():
            raise KeyError

        self._neighbors[neighbor.identifier] = neighbor

        try:
            neighbor.connect(self)
        except KeyError:
            pass

    def find(self, questioners: set[str], subject: str) -> INode:
        if (immediate_node := self._neighbors.get(subject)) is not None:
            return immediate_node

        new_questioners = questioners | {self.identifier}
        for immediate in [v for k, v in self._neighbors.items() if k not in questioners]:
            if (node := immediate.find(new_questioners, subject)) is not None:
                return node

        raise KeyError

    def __str__(self) -> str:
        return f'{self.identifier} ({self.public_key}) - {self.channel[1]}:{self.channel[2]} ({self.channel[0]})'

In [21]:
# test
first_node = Node('first_node', 'ex-pub-key-1', (ChannelType.HTTP, 'IP1', 'PORT1'))
secnd_node = Node('secnd_node', 'ex-pub-key-2', (ChannelType.GRPC, 'IP2', 'PORT1'))
third_node = Node('third_node', 'ex-pub-key-3', (ChannelType.GRPC, 'IP2', 'PORT2'))
furth_node = Node('furth_node', 'ex-pub-key-4', (ChannelType.HTTP, 'IP1', 'PORT2'))
fifth_node = Node('fifth_node', 'ex-pub-key-5', (ChannelType.HTTP, 'IP1', 'PORT3'))
sixth_node = Node('sixth_node', 'ex-pub-key-6', (ChannelType.GRPC, 'IP1', 'PORT4'))
svnth_node = Node('svnth_node', 'ex-pub-key-6', (ChannelType.HTTP, 'IP2', 'PORT3'))

first_node.connect(secnd_node)
first_node.connect(third_node)
first_node.connect(fifth_node)
third_node.connect(furth_node)
secnd_node.connect(sixth_node)

print('first: ', ', '.join(first_node._neighbors.keys()))
print('secnd: ', ', '.join(secnd_node._neighbors.keys()))
print('third: ', ', '.join(third_node._neighbors.keys()))
print('furth: ', ', '.join(furth_node._neighbors.keys()))
print('fifth: ', ', '.join(fifth_node._neighbors.keys()))
print('sixth: ', ', '.join(sixth_node._neighbors.keys()))
print('svnth: ', ', '.join(svnth_node._neighbors.keys()))

print('query first on sixth: ', sixth_node.find({'svnth_node'}, 'first_node'))
print('query sixth on furth: ', furth_node.find({'svnth_node'}, 'sixth_node'))
print('query svnth on first: ', first_node.find({'secnd_node'}, 'svnth_node'))

first:  secnd_node, third_node, fifth_node
secnd:  first_node, sixth_node
third:  first_node, furth_node
furth:  third_node
fifth:  first_node
sixth:  secnd_node
svnth:  
query first on sixth:  first_node (ex-pub-key-1) - IP1:PORT1 (ChannelType.HTTP)
query sixth on furth:  sixth_node (ex-pub-key-6) - IP1:PORT4 (ChannelType.GRPC)


KeyError: 